<a href="https://colab.research.google.com/github/tejomaygadgil/cgpos/blob/nn/notebooks/colab/1_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install wandb
import wandb
wandb.login()

In [ ]:
!git clone -b nn https://github.com/tejomaygadgil/cgpos.git
!pip install git+https://github.com/jtauber/greek-accentuation.git beta-code
%cd /content/cgpos
!make process_ft_data
!python src/train.py setup pt_cloud

In [ ]:
!git pull

In [ ]:
!python src/train.py fine_tune